## 0. Make Model

#사진데이터 전처리 일반적인 얼굴을 구함
1. Face Detection
2. Landmark Detection
3. 배경은 투명인 마스크이미지를 landmark 기준으로 얼굴에 붙임(눈과 코를 기준)
4. 마스크를 안쓴 이미지 -> 마스크를 쓴 이미지
   다양한 이미지 사용. 사람들 이미지수를 늘리는지해서 Dataset을 만듦

5. 모델구조(전이학습)
2개의 아웃풋[0 , 0]
softmax를 통해 마스크썻으면 1,0 마스크 안썻으면 0,1 
cross entropy

In [1]:
cd ./maskdata/

C:\Users\kimhy\K-Digital MLDL\Semi-project3\4팀\maskdata


In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
 
# 마스크데이터 
path_dir1 = './Non_Mask/'
path_dir2 = './Mask/'

# path에 존재하는 파일 목록 가져오기
file_list1 = os.listdir(path_dir1)
file_list2 = os.listdir(path_dir2)
 
file_list1_num = len(file_list1)
file_list2_num = len(file_list2)
 
file_num = file_list1_num + file_list2_num
 
 
#이미지 전처리
# https://ivo-lee.tistory.com/91
num = 0;
all_img = np.float32(np.zeros((file_num, 224, 224, 3))) # zeros는 0으로 가득 찬 array를 생성합니다. # np.zeros(shape, dtype, order) dtype default = float64, order는 저장할때 다차원 순서
all_label = np.float64(np.zeros((file_num, 1))) #0아니면 1 할거라서 64?
 
for img_name in file_list1:
    img_path = path_dir1 + img_name
    img = load_img(img_path, target_size=(224, 224))
    
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    all_img[num, :, :, :] = x
    
    all_label[num] = 0 # nomask
    num = num + 1
 
for img_name in file_list2:
    img_path = path_dir2+img_name
    img = load_img(img_path, target_size=(224, 224))
    
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    all_img[num, :, :, :] = x
    
    all_label[num] = 1 # mask
    num = num + 1
 
 
# 데이터셋 섞기(적절하게 훈련되게 하기 위함) 
n_elem = all_label.shape[0]
indices = np.random.choice(n_elem, size=n_elem, replace=False)
 
all_label = all_label[indices]
all_img = all_img[indices]
 
 
# 훈련셋 테스트셋 분할
num_train = int(np.round(all_label.shape[0]*0.8))
num_test = int(np.round(all_label.shape[0]*0.2))
 
train_img = all_img[0:num_train, :, :, :]
test_img = all_img[num_train:, :, :, :] 
 
train_label = all_label[0:num_train]
test_label = all_label[num_train:]
 
 
# create the base pre-trained model
IMG_SHAPE = (224, 224, 3)
 
base_model = ResNet50(input_shape=IMG_SHAPE, weights='imagenet', include_top=False)
base_model.trainable = False
base_model.summary()
print("Number of layers in the base model: ", len(base_model.layers))
 
flatten_layer = Flatten()
dense_layer1 = Dense(128, activation='relu')
bn_layer1 = BatchNormalization()
dense_layer2 = Dense(1, activation=tf.nn.sigmoid)
 
model = Sequential([
        base_model,
        flatten_layer,
        dense_layer1,
        bn_layer1,
        dense_layer2,
        ])
 
base_learning_rate = 0.001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()
 
model.fit(train_img, train_label, epochs=10, batch_size=16, validation_data = (test_img, test_label))
 
 

# save model
model.save("model.h5")
 
print("Saved model to disk") 